<a href="https://colab.research.google.com/github/eshhacb/Election-Sentiment-Classifier/blob/main/USA_Election_Syntiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
trump_reviews = pd.read_csv("Trumpall2.csv")
biden_reviews = pd.read_csv("Bidenall2.csv")

In [ ]:
trump_reviews.head(3)

,user,text
0,manny_rosen,@sanofi please tell us how many shares the Cr...
1,osi_abdul,"https://t.co/atM98CpqF7 Like, comment, RT #P..."
2,Patsyrw,Your AG Barr is as useless &amp; corrupt as y...


In [ ]:
biden_reviews.head(3)

,user,text
0,MarkHodder3,@JoeBiden And we’ll find out who won in 2026...
1,K87327961G,@JoeBiden Your Democratic Nazi Party cannot be...
2,OldlaceA,@JoeBiden So did Lying Barr


# Sentiment Analysis
Now, I will get started with sentiment analysis. I will do it by using the Textblob package in Python. Here I will use this package to perform simple text classification in either positive or negative on the basis of sentiment analysis:

1 Polarity refers to the sentiment or emotion expressed in the text, ranging from negative to positive.


2 Subjectivity,suggests that the text contains some subjects or opinions, measures the extent to which the text is subjective or opinionated rather than objective

In [ ]:
textblob1 = TextBlob(trump_reviews['text'][10])
textblob1.sentiment
textblob2 = TextBlob(biden_reviews['text'][500])
textblob2.sentiment

Sentiment(polarity=0.6, subjectivity=0.9)

In [ ]:
def find_pol(review):
    return TextBlob(review).sentiment.polarity

In [ ]:
trump_reviews["Sentiment Polarity"] = trump_reviews["text"].apply(find_pol)
biden_reviews["Sentiment Polarity"] = biden_reviews["text"].apply(find_pol)

In [ ]:
trump_reviews.head(3)

,user,text,Sentiment Polarity
0,manny_rosen,@sanofi please tell us how many shares the Cr...,0.05
1,osi_abdul,"https://t.co/atM98CpqF7 Like, comment, RT #P...",0.00
2,Patsyrw,Your AG Barr is as useless &amp; corrupt as y...,-0.50


In [ ]:
biden_reviews.head(3)

,user,text,Sentiment Polarity
0,MarkHodder3,@JoeBiden And we’ll find out who won in 2026...,0.0
1,K87327961G,@JoeBiden Your Democratic Nazi Party cannot be...,0.0
2,OldlaceA,@JoeBiden So did Lying Barr,0.0


# Sentiment Polarity on Both the candidates:
Now I will add a new attribute in both the datasets by the name of “Expression Label”:                                        
1 polarity > 0 :: positive                                                               
2 polarity < 0 :: negative                                                     
3 Polarity == 0 :: neutral

In [ ]:
trump_reviews["Expression Label"] = np.where(trump_reviews["Sentiment Polarity"]>0, "positive", "negative")
trump_reviews["Expression Label"][trump_reviews["Sentiment Polarity"]==0]="Neutral"
print(trump_reviews.tail())

biden_reviews["Expression Label"] = np.where(biden_reviews["Sentiment Polarity"]>0, "positive", "negative")
biden_reviews["Expression Label"][biden_reviews["Sentiment Polarity"]==0]="Neutral"
print(biden_reviews.tail())

                 user                                               text  \
2783          4diva63  @realDonaldTrump For the 1/100 time, absentee ...   
2784         hidge826  @realDonaldTrump If you’re so scared of losing...   
2785     SpencerRossy  @realDonaldTrump I rarely get involved with fo...   
2786  ScoobyMcpherson  @realDonaldTrump This is the moment when Trump...   
2787          bjklinz     @realDonaldTrump I’m sorry, Donald. No. #POTUS   

      Sentiment Polarity Expression Label  
2783               0.000          Neutral  
2784               0.000          Neutral  
2785               0.225         positive  
2786               0.000          Neutral  
2787              -0.500         negative  
             user                                               text  \
2535    meryn1977  @JoeBiden You'll just try to calm those waters...   
2536  BSNelson114  @JoeBiden 96 days 96 dias #VoteJoeBiden2020  #...   
2537     KenCapel  @JoeBiden YOU THINK YOU CAN DO THAT??? YOU C

/tmp/ipython-input-883805447.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  trump_reviews["Expression Label"][trump_reviews["Sentiment Polarity"]==0]="Neutral"
/tmp/ipython-input-883805447.py:2: SettingWithCopyWarning: 
A value is tryin

# Drop all the tweets with neutral polarity

In [ ]:
reviews1 = trump_reviews[trump_reviews['Sentiment Polarity'] == 0.0000]
print(reviews1.shape)

cond1=trump_reviews['Sentiment Polarity'].isin(reviews1['Sentiment Polarity'])
trump_reviews.drop(trump_reviews[cond1].index, inplace = True)
print(trump_reviews.shape)

(1464, 4)
(1324, 4)


In [ ]:
reviews2 = biden_reviews[biden_reviews['Sentiment Polarity'] == 0.0000]
print(reviews2.shape)

cond2=biden_reviews['Sentiment Polarity'].isin(reviews1['Sentiment Polarity'])
biden_reviews.drop(biden_reviews[cond2].index, inplace = True)
print(biden_reviews.shape)

(1509, 4)
(1031, 4)


# balance both the datasets

In [ ]:
# Donald Trump
np.random.seed(10)
remove_n =324
drop_indices = np.random.choice(trump_reviews.index, remove_n, replace=False)
df_subset_trump = trump_reviews.drop(drop_indices)
print(df_subset_trump.shape)
# Joe Biden
np.random.seed(10)
remove_n =31
drop_indices = np.random.choice(biden_reviews.index, remove_n, replace=False)
df_subset_biden = biden_reviews.drop(drop_indices)
print(df_subset_biden.shape)

(1000, 4)
(1000, 4)


# predict the US Elections, by analyzing the number of positive and negative sentiments in both the accounts:

In [ ]:
count_1 = df_subset_trump.groupby('Expression Label').count()
print(count_1)

negative_per1 = (count_1['Sentiment Polarity'][0]/1000)*10
positive_per1 = (count_1['Sentiment Polarity'][1]/1000)*100

count_2 = df_subset_biden.groupby('Expression Label').count()
print(count_2)

negative_per2 = (count_2['Sentiment Polarity'][0]/1000)*100
positive_per2 = (count_2['Sentiment Polarity'][1]/1000)*100



                  user  text  Sentiment Polarity
Expression Label                                
negative           449   449                 449
positive           551   551                 551
                  user  text  Sentiment Polarity
Expression Label                                
negative           393   393                 393
positive           607   607                 607


/tmp/ipython-input-3657419719.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  negative_per1 = (count_1['Sentiment Polarity'][0]/1000)*10
/tmp/ipython-input-3657419719.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  positive_per1 = (count_1['Sentiment Polarity'][1]/1000)*100
/tmp/ipython-input-3657419719.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  negative_per2 = (count_2['Sentiment Polarity'][0]/1000)*100
/tmp/ipyth

In [ ]:
Politicians = ['Joe Biden', 'Donald Trump']
lis_pos = [positive_per1, positive_per2]
lis_neg = [negative_per1, negative_per2]

fig = go.Figure(data=[
    go.Bar(name='Positive', x=Politicians, y=lis_pos),
    go.Bar(name='Negative', x=Politicians, y=lis_neg)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()